### SYNTHETIC IMAGE GENERATION USING GENERATIVE ADVERSIAL NETWORK

BY UBONG BEN



Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/2a/b7/a3cf5fd40334b9785cc83ee0c96b50603026eb3aa70210a33729018e7029/torch-2.3.0-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.3.0-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for mkl<=2021.4.0,>=2021.1.1 from https://files.pythonhosted.org/packages/fe/1c/5f6dbf18e8b73e0a5472466f0ea8d48ce9efae39bd2ff38cebf8dce61259/mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Obtaining dependency information for intel-ope

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/12/c2/7c89c62f2b0a606070aa7cdb8af8af0c905562942778ebdd77600642c3b9/torchvision-0.18.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 87.5 kB/s eta 0:00:14
   - -------------------------------------- 0.0/1.2 MB 87.5 kB/s eta 0:00:14
   - -------------------------------------- 0.0/1.2 MB 87.5 kB/s eta

In [2]:
#Libraries

import torch
from torchvision.datasets import MNIST 
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import numpy as np

ModuleNotFoundError: No module named 'torch'

function to transform and load the data with Torch

In [ ]:

def get_dl(batchsize):

    #  Transform the data as Tensors
    train_transforms = transforms.Compose([transforms.ToTensor()])
    
    # Collect NNIST data and transform it to tensors
    train_data = MNIST(root="./train.",train=True,download=True,transform=train_transforms)
    test_data = MNIST(root="./test.",train=True,download=True,transform=train_transforms)
    
    # Load  data
    train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=False,drop_last=True)
    test_loader = DataLoader(test_data,batch_size=batchsize,shuffle=False,drop_last=True)
    
    # Return train and test data 
    return train_loader,test_loader

GENRATIVE ADVERSIAL NETWORK

In [ ]:
class Generator(nn.Module):
    def __init__(self,batch_size,input_dim):
        super().__init__()
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim,128)
        self.LRelu = nn.LeakyReLU()
        self.fc2 = nn.Linear(128,1*28*28)
        self.tanH = nn.Tanh()

    # forward propogation
    def forward(self,x):
        layer1 = self.LRelu(self.fc1(x))
        layer2 = self.tanH(self.fc2(layer1))
        out =layer2.view(self.batch_size,1,28,28)
        return out


Discriminator network

In [ ]:
class Discriminator(nn.Module):
    def __init__(self,batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.fc1 = nn.Linear(1*28*28,128)
        self.LReLu = nn.LeakyReLU()
        self.fc2 = nn.Linear(128,1)
        self.SigmoidL = nn.Sigmoid()

    # Forward propogation
    def forward(self,x):
        flat = x.view(self.batch_size,-1)
        layer1 = self.LReLu(self.fc1(flat))
        out = self.SigmoidL(self.fc2(layer1))
        return out.view(-1,1).squeeze(1)

Function to train the model

In [ ]:
def train_model(no_of_epochs,disc,gen,optimD,optimG,dataloaders,loss_fn,input_size,batch_size):
    """
    disc: Discriminator model
    gen: Generator model
    optimD: Optimizer for Discriminator
    optimG: Optimizer for Generator
    """
    
    # set device as CUDA
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    reall = 1  # real label
    fakel = 0  # fake label
    #running each epoch
    for epoch in range(no_of_epochs):
        print('Epoch {}/{}'.format(epoch+1,no_of_epochs))
        running_loss_D = 0
        running_loss_G = 0
        for phase in ["train"]:
            #getting input and label from dataloader
            for inputs, _ in dataloaders[phase]:
                inputs = inputs.to(device)
                #converting labels into torch with proper size as per the batch size
                real_label = torch.full((batch_size,),reall,dtype=inputs.dtype,device=device)
                fake_label = torch.full((batch_size,),fakel,dtype=inputs.dtype,device=device)

                optimD.zero_grad()
                #output from discriminator
                output =disc(inputs)
                #Discriminator real loss
                # Compairing output label with real label which is loss
                D_real_loss = loss_fn(output,real_label)
                D_real_loss.backward()
                
                #random torch tensor as a noise data
                noise = torch.randn(batch_size,input_size,device=device)
                #passing noise throgh generator to get fake image
                fake = gen(noise)
                #passing fake image through discriminator with detaching(not passing gradient)
                output = disc(fake.detach())
                
                #Discriminator fake loss
                D_fake_loss = loss_fn(output,fake_label)
                #back propogation
                D_fake_loss.backward()

                # total loss for Discriminator
                Disc_loss = D_real_loss+D_fake_loss
                running_loss_D = running_loss_D+Disc_loss
                optimD.step()

                optimG.zero_grad()
                #passing fake image obtained from generator to discriminator
                output = disc(fake)
                #getting generator loss by giving fake image as input but giving real label
                Gen_loss = loss_fn(output,real_label)
                running_loss_G = running_loss_G + Gen_loss
                #backpropogation
                Gen_loss.backward()
                optimG.step()
        print("Discriminator Loss : {}".format(running_loss_D))
        print("Generator Loss : {}".format(running_loss_G))

In [ ]:
#fixing seed
torch.manual_seed(4)
batch_size = 128
no_of_epochs = 5
#input sixe for the latent variable
input_size = 100
#getting train and test data loader and putting them inside the dictionary
train_loader,test_loader = get_dl(batch_size)
dl = {}
dl['train'] = train_loader
dl['valid'] = test_loader
#discriminator model
disc  = Discriminator(batch_size)
#generator model
gen = Generator(batch_size,input_size)
#Optimizer for Discriminator
optimD  = torch.optim.Adam(disc.parameters(),lr=0.001,weight_decay=1e-05)
#Optimizer for Generator
optimG  = torch.optim.Adam(gen.parameters(),lr=0.001,weight_decay=1e-05)
#Binary cross entroy as loss function
loss_fn = torch.nn.BCELoss()
#moving models to the device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
disc.to(device)
gen.to(device)
#training the model
train_model(no_of_epochs,disc,gen,optimD,optimG,dl,loss_fn,input_size,batch_size)

In [ ]:
import matplotlib.pyplot as plt
#function to plot the image
def show_image(img):
  #converting imgae from tensor to numpy
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg,(1,2,0)))

In [ ]:
import torchvision
random_noise = torch.randn(128,input_size,device=device)
#generationg fake image from random noise
fake = gen(random_noise)
fake = fake.cpu()
#plotting the fake image
fig,ax = plt.subplots(figsize=(20,8.5))
show_image(torchvision.utils.make_grid(fake[0:50],10,5))
plt.show()